In [2]:
import sympy as sym

In [3]:
from sympy.printing.ccode import C99CodePrinter
from sympy.printing.codeprinter import Assignment 

In [4]:
import pprint
from sympy import *

In [5]:
sym.init_printing()

In [130]:
class MyCodePrinter(C99CodePrinter):
    def __init__(self, *args, **kwargs):
        super(C99CodePrinter, self).__init__(*args, **kwargs)
    def _print_tuple(self, expr):
        lines = []
        for item in expr:
            if not item.has(Assignment):
                raise ValueError("Tuples must only contain assignment")
            lines.append(self._print(item))
        return '\n'.join(lines)
    
printer = MyCodePrinter(settings = dict(user_functions = {'Mod' : 'fmodr'}))

In [7]:
#printer.doprint(Piecewise)

def scoped_piecewise(cond, ts, fs, assign_tos):
    texprs = tuple([ Assignment(a, e) for e,a in zip(ts, assign_tos) ])
    fexprs = tuple([ Assignment(a, e) for e,a in zip(fs, assign_tos) ])
    return sym.Piecewise((texprs, cond), (fexprs, True))

In [173]:
def mod2pi(x):
    return x % (2*sym.pi)

def dubins_LSL(a, b, d):
    tmp0 = d - sin(a) + sin(b)
    p_sq = 2 + (d*d) - 2 * cos(a - b) + (2*d*(sin(a)-sin(b)))
    tmp1 = atan2( (cos(b)-cos(a)), tmp0 );
    t = mod2pi(-a + tmp1 );
    p = sqrt( p_sq );
    q = mod2pi(b - tmp1 );
    cond = p_sq >= 0
    return t, p, q, cond

def dubins_RSR(a, b, d):
    tmp0 = d-sin(a)+sin(b)
    p_sq = 2 + (d*d) -(2*cos(a - b)) + (2*d*(sin(b)-sin(a)))
    tmp1 = atan2( cos(a)-cos(b), tmp0 )
    t = mod2pi( a - tmp1 );
    p = sqrt( p_sq );
    q = mod2pi( -b + tmp1 );
    cond = GreaterThan(p_sq, 0)
    return t, p, q, cond
    

def dubins_LSR(a, b, d):
    p_sq = -2 + (d*d) + (2*cos(a-b)) + (2*d*(sin(a)+sin(b)));
    p    = sqrt( p_sq );
    tmp2 = atan2( (-cos(a)-cos(b)), (d+sin(a)+sin(b) ) ) - atan2(S(-2), p);
    t    = mod2pi(-a + tmp2);
    q    = mod2pi( -mod2pi(b) + tmp2 );
    cond = p_sq >= 0
    return t, p, q, cond

def dubins_RSL(a, b, d):
    p_sq = (d*d) -2 + (2*cos(a-b)) - (2*d*(sin(a)+sin(b)));
    p    = sqrt( p_sq );
    tmp2 = atan2( (cos(a)+cos(b) ), (d-sin(a)-sin(b) ) ) - atan2(2.0, p);
    t    = mod2pi(a - tmp2);
    q    = mod2pi(b - tmp2);
    cond = p_sq >= 0
    return t, p, q, cond

def dubins_RLR(a, b, d):
    tmp_rlr = (6 - d*d + 2*cos(a-b) + 2*d*(sin(a)-sin(b) )) / S(8);
    p = mod2pi( 2*pi - acos( tmp_rlr ) );
    t = mod2pi(a - atan2( cos(a)-cos(b), d-sin(a)+sin(b) ) + mod2pi(p/S(2)))
    q = mod2pi(a - b - t + mod2pi(p));
    cond = Abs(tmp_rlr) <= 1
    return t, p, q, cond


def dubins_LRL(a, b, d):
    tmp_lrl = (6 - d*d + 2*cos(a-b) + 2*d*(- sin(a) + sin(b) )) / S(8);
    p = mod2pi( 2*pi - acos( tmp_lrl ) );
    t = mod2pi(-a - atan2( cos(a)-cos(b), d+sin(a)-sin(b) ) + p/2.);
    q = mod2pi(mod2pi(b) - a -t + mod2pi(p));
    cond = Abs(tmp_lrl) <= 1
    return t, p, q, cond


a, b = sym.symbols('a b', real=True)
d = sym.symbols('d', real=True, postive=True)
fs = [dubins_LSL, dubins_RSR, dubins_LSR, dubins_RSL, dubins_RLR, dubins_LRL]
xs = []
for f in fs:
    xs.append( list(f(a, b, d) ) )


In [174]:
a, b = sym.symbols('a b', real=True)
d = sym.symbols('d', real=True, postive=True)
fs = [dubins_LSL, dubins_RSR, dubins_LSR, dubins_RSL, dubins_RLR, dubins_LRL]
gs = [ f(a,b,d) for f in fs]

In [175]:


for f in fs:
    print '------------------'
    t, p, q, cond = f(a, b, d)
    subs = {sin(a) : 'sa', sin(b) : 'sb', cos(a) : 'ca', cos(b) : 'cb', cos(a-b) : 'c_ab'}
    cond = cond.subs(subs)
    t = t.subs(subs)
    p = p.subs(subs)
    q = q.subs(subs)
    
    ys = [t, p, q]
    y = sym.MatrixSymbol('y', 3, 1)
    A = sym.Matrix([q, p, t])
    B = sym.Matrix([0, 0, 0])
    expr1 = Piecewise( (Assignment(y, A), cond), (Assignment(y, B), True)) 
    print printer.doprint(expr1)


------------------
if (-2*c_ab + pow(d, 2) + 2*d*(sa - sb) + 2 >= 0) {
   y[0] = fmodr(b - atan2(-ca + cb, d - sa + sb), 2*M_PI);
   y[1] = sqrt(-2*c_ab + pow(d, 2) + 2*d*(sa - sb) + 2);
   y[2] = fmodr(-a + atan2(-ca + cb, d - sa + sb), 2*M_PI);
}
else {
   y[0] = 0;
   y[1] = 0;
   y[2] = 0;
}
------------------
if (-2*c_ab + pow(d, 2) + 2*d*(-sa + sb) + 2 >= 0) {
   y[0] = fmodr(-b + atan2(ca - cb, d - sa + sb), 2*M_PI);
   y[1] = sqrt(-2*c_ab + pow(d, 2) + 2*d*(-sa + sb) + 2);
   y[2] = fmodr(a - atan2(ca - cb, d - sa + sb), 2*M_PI);
}
else {
   y[0] = 0;
   y[1] = 0;
   y[2] = 0;
}
------------------
if (2*c_ab + pow(d, 2) + 2*d*(sa + sb) - 2 >= 0) {
   y[0] = fmodr(-fmodr(b, 2*M_PI) - atan2(-2, sqrt(2*c_ab + pow(d, 2) + 2*d*(sa + sb) - 2)) + atan2(-ca - cb, d + sa + sb), 2*M_PI);
   y[1] = sqrt(2*c_ab + pow(d, 2) + 2*d*(sa + sb) - 2);
   y[2] = fmodr(-a - atan2(-2, sqrt(2*c_ab + pow(d, 2) + 2*d*(sa + sb) - 2)) + atan2(-ca - cb, d + sa + sb), 2*M_PI);
}
else {
   y[0] = 0;
   y[1]

In [177]:
sym.atan2(0,0)

In [183]:
expr = acos(a+b*d)+b

In [190]:
for se in sym.preorder_traversal(expr):
    if type(se) == acos:
        print se.args
    #print se, type(se)==acos

(a + b*d,)


In [191]:
sqrt(a+b)

In [192]:
sqrt.domain

AttributeError: 'function' object has no attribute 'domain'

In [204]:
sym.simplify(2 + (d*d) - 2 * cos(a - b) + (2*d*(sin(a)-sin(b))))

In [225]:
cx1 = cos(a+pi/2)
cy1 = sin(a+pi/2)
cx2 = d + cos(b+pi/2)
cy2 = sin(b+pi/2)
dx = cx2-cx1
dy = cy2-cy1
print 'p^2 = ', simplify(dx**2+dy**2)

p^2 =  d**2 + 2*d*sin(a) - 2*d*sin(b) - 2*cos(a - b) + 2


In [229]:
simplify(d**2+2*d*sin(a) -2*d*sin(b)+2)